# Function 1: 2D Radiation Detection

Explore the initial 10 (x, y) points, visualize the landscape, and suggest the next input to submit.

- 2D, sparse signal (only proximity yields non-zero reading).
- One hotspot; goal is to find it with limited queries.

## 1. Setup and load data (read-only from initial_data)

In [ ]:
import sys
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
from pathlib import Path

from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel, Matern, WhiteKernel
from scipy.stats import norm
import warnings
warnings.filterwarnings("ignore")


# Project root: works whether you run from repo root or from notebooks/
repo_root = Path.cwd() if (Path.cwd() / "src").exists() else Path.cwd().parent
sys.path.insert(0, str(repo_root))

from src.utils.load_challenge_data import (
    load_function_data,
    load_problem_data_csv,
    save_problem_data_csv,
    assert_not_under_initial_data,
)
from src.optimizers.bayesian.acquisition_functions import (
    entropy_search,
    expected_improvement,
    probability_of_improvement,
    thompson_sampling_sample,
    upper_confidence_bound,
)
from src.utils.plot_utilities import (
    add_colorbar,
    style_axis,
    style_axis_3d,
    style_legend,
    DEFAULT_FONT_SIZE_AXIS,
    DEFAULT_FONT_SIZE_TITLES,
    DEFAULT_EXPORT_DPI,
    DEFAULT_EXPORT_FORMAT,
)
from src.utils.sampling_utils import sample_candidates

# --- Candidate sampling for acquisition maximisation: 'grid', 'lhs', 'sobol', or 'random' ---
# grid: regular lattice; best when n ≈ k^dim (e.g. 2D with 80² points). Deterministic.
# lhs:  Latin Hypercube; one point per bin per dimension. Good space-filling, reproducible with seed.
# sobol: Sobol (quasi-Monte Carlo); low-discrepancy, often better coverage than random. Good default.
# random: i.i.d. uniform; simple but can leave gaps. Use for quick tests.
# Suitability: sobol/lhs for exploration and even coverage; grid when you want a fixed lattice; random only if others unavailable.
CANDIDATE_SAMPLING_METHOD = "lhs"

# --- Plot options; font/export defaults from utils (DEFAULT) ---
IF_SHOW_PLOT = True
IF_EXPORT_PLOT = False
PLOT_EXPORT_DIR = repo_root / "data" / "results"

IF_EXPORT_QUERIES = False
IF_APPEND_DATA = False  # Set True and paste (x1, x2), y in "Append new feedback" cell to add new points to the local dataset

# Set seed for reproducibility
np.random.seed(42)

In [ ]:
# Load: under data/ we use only CSV. observations.csv if present, else initial_data (read-only).
local_dir = repo_root / "data" / "problems" / "function_1"
csv_path = local_dir / "observations.csv"
if csv_path.exists():
    X, y = load_problem_data_csv(csv_path)
    print("Loaded from local CSV (initial + appended):", csv_path)
else:
    X, y = load_function_data(function_id=1)
    print("Loaded from initial_data (read-only). Run append script or 'Append data' after portal feedback.")
print('Dataset info:')
print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")
print(f"X max value: {X.max():.6f}")
print(f"X min value: {X.min():.6f}")
print(f"y max value: {y.max():.6f}")
print(f"y min value: {y.min():.6f}")
# Are we getting better? (goal: maximize y)
best_idx = np.argmax(y)
best_y, best_x_so_far = y[best_idx], X[best_idx]
print(f"\n>>> Best so far: y = {best_y:.6g} at x = ({best_x_so_far[0]:.4f}, {best_x_so_far[1]:.4f}) | n = {len(y)} points. New portal results improve us if the returned y > {best_y:.6g}.")

### Progress: are we getting better?
Left: **y** at each query (order of observations). Right: **best y so far** (cumulative max) — this line only goes up when a new observation improves on the previous best.

In [ ]:
# Progress line plots: are we getting better over time?
import numpy as np
n_obs = len(y)
obs_idx = np.arange(1, n_obs + 1, dtype=float)
best_so_far = np.maximum.accumulate(y)  # running maximum

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 4))
ax1.plot(obs_idx, y, 'o-', color='blue', markersize=5, label='y per observation')
ax1.set_xlabel('Observation index')
ax1.set_ylabel('y')
ax1.set_title('Output y at each query')
ax1.legend()
ax1.grid(True, alpha=0.3)
style_axis(ax1, font_size_axis=DEFAULT_FONT_SIZE_AXIS, font_size_titles=DEFAULT_FONT_SIZE_TITLES)

ax2.plot(obs_idx, best_so_far, 'o-', color='red', markersize=5, label='Best y so far')
ax2.set_xlabel('Observation index')
ax2.set_ylabel('Best y so far')
ax2.set_title('Cumulative best (improvement over time)')
ax2.legend()
ax2.grid(True, alpha=0.3)
style_axis(ax2, font_size_axis=DEFAULT_FONT_SIZE_AXIS, font_size_titles=DEFAULT_FONT_SIZE_TITLES)
plt.tight_layout()
plt.show()

## 2. Visualize the points

**Convention (no mix-up):** `X` has shape (n, 2) with columns **x_1** = `X[:, 0]` and **x_2** = `X[:, 1]`. **y** is the objective value (scalar per row). All 2D plots: horizontal = x_1, vertical = x_2. 3D: floor = (x_1, x_2), height = y.

In [ ]:
# Grid for contour: distance from each point to nearest observation
n_grid = 80
x1g = np.linspace(0, 1, n_grid)
x2g = np.linspace(0, 1, n_grid)
X1g, X2g = np.meshgrid(x1g, x2g)
grid_pts = np.column_stack([X1g.ravel(), X2g.ravel()])
# Uniform-coverage candidates for acquisition maximisation (grid = lattice; or 'lhs' / 'sobol' / 'random')
candidate_pts = sample_candidates(n_grid * n_grid, 2, method=CANDIDATE_SAMPLING_METHOD, seed=42)
# Distance from each grid point to nearest of X (L2, numpy only)
diff = grid_pts[:, None, :] - X[None, :, :]  # (n_grid^2, n_obs, 2)
dists = np.sqrt((diff ** 2).sum(axis=2))
min_dist = np.min(dists, axis=1).reshape(X1g.shape)

# Interpolate y onto grid (IDW) for 3D surface
dist_gx = np.sqrt(((grid_pts[:, None, :] - X[None, :, :]) ** 2).sum(axis=2)) + 1e-12
w = 1.0 / (dist_gx ** 2)
y_grid = (w * y[None, :]).sum(axis=1) / w.sum(axis=1)
Y_grid = y_grid.reshape(X1g.shape)

fig = plt.figure(figsize=(12, 5))
ax1 = fig.add_subplot(121)
ax2 = fig.add_subplot(122)
# Left: scatter of observations colored by y
sc = ax1.scatter(X[:, 0], X[:, 1], c=y, s=80, cmap="inferno", edgecolors="k")
add_colorbar(sc, ax=ax1, label="y", font_size_axis=DEFAULT_FONT_SIZE_AXIS)
# Enumerate observations from first to last (1, 2, ..., n)
for i in range(len(y)):
    ax1.text(X[i, 0] + 0.025, X[i, 1] + 0.025, str(i + 1), fontsize=DEFAULT_FONT_SIZE_AXIS - 1, color="black", ha="left", va="bottom", zorder=10)
style_axis(ax1, xlabel="x_1", ylabel="x_2", title="Function 2: observations (y)", font_size_axis=DEFAULT_FONT_SIZE_AXIS, font_size_titles=DEFAULT_FONT_SIZE_TITLES)
ax1.set_aspect("equal")

# Right: contour of output y (interpolated)
cf = ax2.contourf(X1g, X2g, Y_grid, levels=22, cmap="turbo")
add_colorbar(cf, ax=ax2, label="y (interpolated)", font_size_axis=DEFAULT_FONT_SIZE_AXIS)
ax2.scatter(X[:, 0], X[:, 1], c=y, s=80, cmap="inferno", edgecolors="black", linewidths=1.5, zorder=2, label="points=f(y)")
style_axis(ax2, xlabel="x_1", ylabel="x_2", title="y (interpolated)", font_size_axis=DEFAULT_FONT_SIZE_AXIS, font_size_titles=DEFAULT_FONT_SIZE_TITLES)
ax2.set_aspect("equal")
style_legend(ax2, loc="upper right", font_size_axis=DEFAULT_FONT_SIZE_AXIS)

# For second figure (3D): norm and mappable for output y colour
norm_y = plt.Normalize(Y_grid.min(), Y_grid.max())
sm = cm.ScalarMappable(cmap=cm.inferno, norm=norm_y)
sm.set_array(Y_grid)

if IF_SHOW_PLOT or IF_EXPORT_PLOT:
    # Second figure: 3D plot only (standalone; created for display and/or export)
    fig2 = plt.figure(figsize=(7, 5))
    ax3b = fig2.add_subplot(111, projection="3d")
    facecolors = cm.inferno(norm_y(Y_grid))[:-1, :-1]
    ax3b.plot_surface(X1g, X2g, Y_grid, facecolors=facecolors, rstride=1, cstride=1, shade=False, alpha=0.8)
    ax3b.scatter(X[:, 0], X[:, 1], y, c='black', s=40, edgecolors="k", depthshade=False)
    n_pts = len(y)
    for i in range(n_pts):
        ax3b.text(X[i, 0], X[i, 1], y[i], str(i + 1), fontsize=DEFAULT_FONT_SIZE_AXIS - 2, color="white", ha="center", va="center")
    style_axis_3d(ax3b, xlabel="x_1", ylabel="x_2", zlabel="y", title="y (*interpolated*), colour = y", font_size_axis=DEFAULT_FONT_SIZE_AXIS, font_size_titles=DEFAULT_FONT_SIZE_TITLES)
    add_colorbar(sm, ax=ax3b, label="y", font_size_axis=DEFAULT_FONT_SIZE_AXIS, shrink=0.8, pad=0.08)

plt.tight_layout()
if IF_EXPORT_PLOT:
    out_dir = PLOT_EXPORT_DIR
    assert_not_under_initial_data(out_dir, project_root=repo_root)
    out_dir.mkdir(parents=True, exist_ok=True)
    out_path_2d = out_dir / f"function_2_observations_and_distance_contour.{DEFAULT_EXPORT_FORMAT}"
    fig.savefig(out_path_2d, dpi=DEFAULT_EXPORT_DPI, format=DEFAULT_EXPORT_FORMAT)
    print("Plot saved to", out_path_2d)
    out_path_3d = out_dir / f"function_2_3d_surface_distance_colour.{DEFAULT_EXPORT_FORMAT}"
    fig2.savefig(out_path_3d, dpi=DEFAULT_EXPORT_DPI, format=DEFAULT_EXPORT_FORMAT)
    print("Plot saved to", out_path_3d)
if IF_SHOW_PLOT:
    plt.show()

## 3. Suggest next point to submit, using Bayesian Optimization Methodology

### 3.1 The Probabilistic Surrogate Models

 * **Gaussian Process (GP)** — we compare three kernel choices below; each gives a different prior over functions and uncertainty:

    - **RBF kernel** — Smooth, infinitely differentiable prior; single length-scale. *Differs:* no explicit noise term (fixed small `alpha` for stability).

    - **Matérn kernel (ν=1.5)** — Less smooth than RBF; allows rougher, more "wiggly" surfaces. *Differs:* better when the true function has sharper changes or only finitely many derivatives.

    - **RBF + WhiteKernel** — Same smoothness as RBF but adds a separate noise term. *Differs:* explicitly models observation noise; uncertainty splits into noise vs interpolation.

 * **Random Forest (RF)** — *Different approach:* non-parametric ensemble of trees; no Gaussian prior or native uncertainty (would need e.g. quantile regression for uncertainty). Not implemented below; listed as an alternative surrogate family.


In [ ]:
# Coefficients of the Probabilistic Surrogate Models (used by GP kernels and acquisition functions below)

# --- GP / kernel (surrogate) ---  (F1: sparse hotspot; smaller length scale = more local uncertainty, better for exploration)
CONSTANT_KERNEL_SCALE = 1.0       # signal variance (output scale); typical: 0.1–10
LENGTH_SCALE = 0.15               # smaller = more local variation, uncertainty in gaps (good when hotspot not found yet)
GP_ALPHA = 1e-6                   # diagonal jitter for numerical stability; typical: 1e-8–1e-4
MATERN_NU = 1.5                   # Matérn smoothness (1.5 = once differentiable); typical: 0.5, 1.5, 2.5
WHITE_NOISE_LEVEL = 1e-6         # WhiteKernel observation noise; typical: 1e-8–1e-2

# --- Acquisition functions ---   (F1: favour exploration until we get a non-zero reading)
XI_EI_PI = 0.18                   # prioritise exploration; typical range [0.01–0.2], 0.18 = high end
KAPPA_UCB = 6.0                   # UCB: μ + κσ; typical 2–6, 6 = strong exploration (within logical limit)

**Effect of increasing vs decreasing each coefficient**

| Coefficient | **Increase** | **Decrease** |
|-------------|--------------|--------------|
| **CONSTANT_KERNEL_SCALE** | GP prior allows larger swings in the mean μ(x); predictions can vary more in magnitude. | Prior is tighter around zero; mean predictions stay smaller in scale. |
| **LENGTH_SCALE** | Smoother surrogate: correlation between points extends farther; fewer “wiggles,” more global structure. | Rougher surrogate: correlation drops off quickly; more local variation, can overfit noise. |
| **GP_ALPHA** | More regularization: posterior is smoother, uncertainty (σ) tends to be slightly higher and more stable. | Less regularization: fit tracks data more closely; risk of numerical issues if data are nearly collinear. |
| **MATERN_NU** | Smoother Matérn prior (closer to RBF); fewer sharp turns. | Rougher prior; more wiggly surfaces, better for non-smooth functions. |
| **WHITE_NOISE_LEVEL** | GP attributes more of the variation to observation noise; posterior uncertainty is higher and less “peaked” near data. | GP attributes more to the latent function; lower σ away from data, higher risk of overconfident extrapolation. |
| **XI_EI_PI** (EI, PI, Entropy) | More **exploration**: favours points with higher uncertainty; next query tends to be farther from observed points. | More **exploitation**: favours points near the current best; next query tends to refine the best region. |
| **KAPPA_UCB** | UCB = μ + κσ: more weight on σ → more **exploration** (prefer uncertain regions). | Less weight on σ → more **exploitation** (prefer high μ). |

In [ ]:
# Gaussian Process Regression (run after visualization cell so X1g, grid_pts exist)
# Coefficients from cell above: CONSTANT_KERNEL_SCALE, LENGTH_SCALE, GP_ALPHA, MATERN_NU, WHITE_NOISE_LEVEL
kernel_RBF = ConstantKernel(CONSTANT_KERNEL_SCALE) * RBF(length_scale=LENGTH_SCALE)
gp_RBF = GaussianProcessRegressor(kernel=kernel_RBF, alpha=GP_ALPHA)
gp_RBF.fit(X, y)
mu_gp_RBF, sigma_gp_RBF = gp_RBF.predict(grid_pts, return_std=True)
mu_gp_RBF = mu_gp_RBF.reshape(X1g.shape)
sigma_gp_RBF = sigma_gp_RBF.reshape(X1g.shape)

# Matern: same length_scale; MATERN_NU gives rougher prior than RBF
kernel_Matern = ConstantKernel(CONSTANT_KERNEL_SCALE) * Matern(length_scale=LENGTH_SCALE, nu=MATERN_NU)
gp_Matern = GaussianProcessRegressor(kernel=kernel_Matern, alpha=GP_ALPHA)
gp_Matern.fit(X, y)
mu_gp_Matern, sigma_gp_Matern = gp_Matern.predict(grid_pts, return_std=True)
mu_gp_Matern = mu_gp_Matern.reshape(X1g.shape)
sigma_gp_Matern = sigma_gp_Matern.reshape(X1g.shape)

# RBF + WhiteKernel: WHITE_NOISE_LEVEL for observation noise
kernel_RBF_noise = ConstantKernel(CONSTANT_KERNEL_SCALE) * RBF(length_scale=LENGTH_SCALE) + WhiteKernel(noise_level=WHITE_NOISE_LEVEL)
gp_RBF_noise = GaussianProcessRegressor(kernel=kernel_RBF_noise, alpha=GP_ALPHA)
gp_RBF_noise.fit(X, y)
mu_gp_RBF_noise, sigma_gp_RBF_noise = gp_RBF_noise.predict(grid_pts, return_std=True)
mu_gp_RBF_noise = mu_gp_RBF_noise.reshape(X1g.shape)
sigma_gp_RBF_noise = sigma_gp_RBF_noise.reshape(X1g.shape)

# Predict at candidate_pts for acquisition maximisation (uniform-coverage method from sampling_utils)
mu_cand_RBF, sigma_cand_RBF = gp_RBF.predict(candidate_pts, return_std=True)
mu_cand_Matern, sigma_cand_Matern = gp_Matern.predict(candidate_pts, return_std=True)

In [ ]:
# Plot all three GPs: mean (left) and std (right) per kernel
fig, axes = plt.subplots(3, 2, figsize=(10, 12))
kernels_info = [
    (mu_gp_RBF, sigma_gp_RBF, "RBF"),
    (mu_gp_Matern, sigma_gp_Matern, "Matérn (ν=1.5)"),
    (mu_gp_RBF_noise, sigma_gp_RBF_noise, "RBF + WhiteKernel"),
]
for i, (mu, sig, name) in enumerate(kernels_info):
    ax_mu, ax_sig = axes[i, 0], axes[i, 1]
    cf1 = ax_mu.contourf(X1g, X2g, mu, levels=20, cmap="viridis")
    add_colorbar(cf1, ax=ax_mu, label="μ(x)", font_size_axis=DEFAULT_FONT_SIZE_AXIS)
    ax_mu.scatter(X[:, 0], X[:, 1], c="red", s=60, edgecolors="k")
    style_axis(ax_mu, xlabel="x_1", ylabel="x_2", title=f"{name} — mean", font_size_axis=DEFAULT_FONT_SIZE_AXIS, font_size_titles=DEFAULT_FONT_SIZE_TITLES)
    ax_mu.set_aspect("equal")
    cf2 = ax_sig.contourf(X1g, X2g, sig, levels=20, cmap="rainbow")
    add_colorbar(cf2, ax=ax_sig, label="σ(x)", font_size_axis=DEFAULT_FONT_SIZE_AXIS)
    ax_sig.scatter(X[:, 0], X[:, 1], c="red", s=60, edgecolors="k")
    style_axis(ax_sig, xlabel="x_1", ylabel="x_2", title=f"{name} — std", font_size_axis=DEFAULT_FONT_SIZE_AXIS, font_size_titles=DEFAULT_FONT_SIZE_TITLES)
    ax_sig.set_aspect("equal")
axes[0, 0].set_xlabel("x_1", fontsize=DEFAULT_FONT_SIZE_AXIS)
axes[1, 0].set_xlabel("x_1", fontsize=DEFAULT_FONT_SIZE_AXIS)
axes[2, 0].set_xlabel("x_1", fontsize=DEFAULT_FONT_SIZE_AXIS)
axes[2, 1].set_xlabel("x_1", fontsize=DEFAULT_FONT_SIZE_AXIS)
plt.tight_layout()
if IF_EXPORT_PLOT:
    out_dir = PLOT_EXPORT_DIR
    assert_not_under_initial_data(out_dir, project_root=repo_root)
    out_dir.mkdir(parents=True, exist_ok=True)
    out_path = out_dir / f"function_1_gp_three_kernels.{DEFAULT_EXPORT_FORMAT}"
    fig.savefig(out_path, dpi=DEFAULT_EXPORT_DPI, format=DEFAULT_EXPORT_FORMAT)
    print("Plot saved to", out_path)
if IF_SHOW_PLOT:
    plt.show()  

### 3.2 The Acquisition Functions $\alpha$(x)

* Expected Improvement (EI)

* Upper Confidence Bound (UCB)

* Probability of Improvement (PI)

* Thompson Sampling

* Entropy Search

In [ ]:
# ACQUISITION FUNCTION(s) — coefficients from "Coefficients of the Probabilistic Surrogate Models" cell (XI_EI_PI, KAPPA_UCB)

# --- EXPECTED IMPROVEMENT: EI = E[max(f(x) − f(x⁺), 0)] ---
# Acquisition maximised over candidate_pts (uniform coverage: CANDIDATE_SAMPLING_METHOD, e.g. 'grid')
y_best_EI = y.max()  # best observed value so far (EI baseline)
EI_RBF = expected_improvement(mu_cand_RBF, sigma_cand_RBF, y_best_EI, xi=XI_EI_PI)
x_best_EI_RBF = candidate_pts[np.argmax(EI_RBF)]
EI_Matern = expected_improvement(mu_cand_Matern, sigma_cand_Matern, y_best_EI, xi=XI_EI_PI)
x_best_EI_Matern = candidate_pts[np.argmax(EI_Matern)]


# --- UPPER CONFIDENCE BOUND: UCB = μ(x) + κσ(x) ---
UCB_RBF = upper_confidence_bound(mu_cand_RBF, sigma_cand_RBF, kappa=KAPPA_UCB)
x_best_UCB_RBF = candidate_pts[np.argmax(UCB_RBF)]
UCB_Matern = upper_confidence_bound(mu_cand_Matern, sigma_cand_Matern, kappa=KAPPA_UCB)
x_best_UCB_Matern = candidate_pts[np.argmax(UCB_Matern)]


# --- PROBABILITY OF IMPROVEMENT: PI = P(f(x) > f(x⁺)) ---
PI_RBF = probability_of_improvement(mu_cand_RBF, sigma_cand_RBF, y_best_EI, xi=XI_EI_PI)
x_best_PI_RBF = candidate_pts[np.argmax(PI_RBF)]
PI_Matern = probability_of_improvement(mu_cand_Matern, sigma_cand_Matern, y_best_EI, xi=XI_EI_PI)
x_best_PI_Matern = candidate_pts[np.argmax(PI_Matern)]


# --- THOMPSON SAMPLING: sample from posterior, then maximize the sample ---
sample_thompson_RBF = thompson_sampling_sample(mu_cand_RBF, sigma_cand_RBF)
x_thomson_RBF = candidate_pts[np.argmax(sample_thompson_RBF)]
sample_thompson_Matern = thompson_sampling_sample(mu_cand_Matern, sigma_cand_Matern)
x_thomson_Matern = candidate_pts[np.argmax(sample_thompson_Matern)]


# --- ENTROPY SEARCH: proxy −σ(x) (favor reducing uncertainty) ---
ES_RBF = entropy_search(mu_cand_RBF, sigma_cand_RBF, y_best_EI, xi=XI_EI_PI)
x_entropy_RBF = candidate_pts[np.argmax(ES_RBF)]
ES_Matern = entropy_search(mu_cand_Matern, sigma_cand_Matern, y_best_EI, xi=XI_EI_PI)
x_entropy_Matern = candidate_pts[np.argmax(ES_Matern)]


print(f'Next point (EI) with RBF kernel and xi = {XI_EI_PI}: x_best_EI = ({x_best_EI_RBF[0]:.2f}, {x_best_EI_RBF[1]:.2f}) | current best observed y = {y_best_EI:.2f}')
print(f'Next point (EI) with Matérn kernel and xi = {XI_EI_PI}: x_best_EI = ({x_best_EI_Matern[0]:.2f}, {x_best_EI_Matern[1]:.2f}) | current best observed y = {y_best_EI:.2f}')
print(f'Next point (UCB) with RBF kernel and κ = {KAPPA_UCB}: x_best_UCB = ({x_best_UCB_RBF[0]:.2f}, {x_best_UCB_RBF[1]:.2f}) | current best observed y = {y_best_EI:.2f}')
print(f'Next point (UCB) with Matérn kernel and κ = {KAPPA_UCB}: x_best_UCB = ({x_best_UCB_Matern[0]:.2f}, {x_best_UCB_Matern[1]:.2f}) | current best observed y = {y_best_EI:.2f}')
print(f'Next point (PI) with RBF kernel and xi = {XI_EI_PI}: x_best_PI = ({x_best_PI_RBF[0]:.2f}, {x_best_PI_RBF[1]:.2f}) | current best observed y = {y_best_EI:.2f}')
print(f'Next point (PI) with Matérn kernel and xi = {XI_EI_PI}: x_best_PI = ({x_best_PI_Matern[0]:.2f}, {x_best_PI_Matern[1]:.2f}) | current best observed y = {y_best_EI:.2f}')
print(f'Next point (Thompson) with RBF kernel: x_thomson = ({x_thomson_RBF[0]:.2f}, {x_thomson_RBF[1]:.2f}) | current best observed y = {y_best_EI:.2f}')
print(f'Next point (Thompson) with Matérn kernel: x_thomson = ({x_thomson_Matern[0]:.2f}, {x_thomson_Matern[1]:.2f}) | current best observed y = {y_best_EI:.2f}')
print(f'Next point (Entropy) with RBF kernel: x_entropy = ({x_entropy_RBF[0]:.2f}, {x_entropy_RBF[1]:.2f}) | current best observed y = {y_best_EI:.2f}')
print(f'Next point (Entropy) with Matérn kernel: x_entropy = ({x_entropy_Matern[0]:.2f}, {x_entropy_Matern[1]:.2f}) | current best observed y = {y_best_EI:.2f}')

# Kernel selection sensitivity: Delta = (RBF suggestion) - (Matern suggestion)
delta_EI = x_best_EI_RBF - x_best_EI_Matern
delta_UCB = x_best_UCB_RBF - x_best_UCB_Matern
delta_PI = x_best_PI_RBF - x_best_PI_Matern
delta_Thompson = x_thomson_RBF - x_thomson_Matern
delta_Entropy = x_entropy_RBF - x_entropy_Matern
print('\nKernel selection sensitivity (Delta = RBF - Matern):')
print(f'  EI:       Delta: x_1 = {delta_EI[0]:.2f}, x_2 = {delta_EI[1]:.2f}')
print(f'  UCB:      Delta: x_1 = {delta_UCB[0]:.2f}, x_2 = {delta_UCB[1]:.2f}')
print(f'  PI:       Delta: x_1 = {delta_PI[0]:.2f}, x_2 = {delta_PI[1]:.2f}')
print(f'  Thompson: Delta: x_1 = {delta_Thompson[0]:.2f}, x_2 = {delta_Thompson[1]:.2f}')
print(f'  Entropy:  Delta: x_1 = {delta_Entropy[0]:.2f}, x_2 = {delta_Entropy[1]:.2f}')
print(50*'-')


In [ ]:
# Sanity checks: (0,0) suggestions and low sigma (degenerate EI/PI)
def _is_near_zero(x, tol=1e-6):
    x = np.asarray(x).ravel()
    return x.size >= 2 and np.all(np.abs(x) < tol)

sigma_RBF_max = np.nanmax(sigma_gp_RBF) if sigma_gp_RBF.size else 0.0
sigma_Matern_max = np.nanmax(sigma_gp_Matern) if sigma_gp_Matern.size else 0.0
SIGMA_WARN = 1e-6
if sigma_RBF_max < SIGMA_WARN or sigma_Matern_max < SIGMA_WARN:
    print(f"⚠ GP uncertainty very low: sigma_RBF_max = {sigma_RBF_max:.2e}, sigma_Matern_max = {sigma_Matern_max:.2e}. EI/PI can be degenerate (suggest 0,0).")

suggestions = [
    (x_best_EI_RBF, 'EI RBF'), (x_best_EI_Matern, 'EI Matern'),
    (x_best_UCB_RBF, 'UCB RBF'), (x_best_UCB_Matern, 'UCB Matern'),
    (x_best_PI_RBF, 'PI RBF'), (x_best_PI_Matern, 'PI Matern'),
    (x_thomson_RBF, 'Thompson RBF'), (x_thomson_Matern, 'Thompson Matern'),
    (x_entropy_RBF, 'Entropy RBF'), (x_entropy_Matern, 'Entropy Matern'),
]
near_zero = [name for x, name in suggestions if _is_near_zero(x)]
if near_zero:
    print(f"Suggestions at or near (0,0): {', '.join(near_zero)}. Consider using baseline (exploit/explore) or Thompson.")
else:
    print("No acquisition suggestion at (0,0); checks OK.")


In [ ]:
# Baseline: suggest next x without GP (compare later with acquisition-function suggestions).
# Why exploit? Function 1 has one sparse hotspot — best observed y is near that peak, so a small
# random step from best_x refines the search locally. Explore (random in [0,1]²) would waste
# queries in empty regions. Bounds low/high + margin are for optional use; we clip to [0,1] anyway.
rng = np.random.default_rng(42)
best_idx = np.argmax(y)
best_x = X[best_idx].copy()

# Infer bounds from data (or use [0,1]^2 from config)
low = X.min(axis=0)
high = X.max(axis=0)
# Widen slightly so we can suggest points near the edges
margin = 0.05
low = np.clip(low - margin, 0, 1)
high = np.clip(high + margin, 0, 1)

# Option A: small step from best (exploit)
step = 0.1
next_x_exploit = best_x + rng.uniform(-step, step, size=2)
next_x_exploit = np.clip(next_x_exploit, 0, 1)

# Option B: random in domain (explore)
next_x_explore = rng.uniform(0, 1, size=2)

# Option C: point with highest distance to nearest observation (on candidate_pts, same uniform coverage)
min_dist_cand = np.min(np.sqrt(((candidate_pts[:, None, :] - X[None, :, :]) ** 2).sum(axis=2)), axis=1)
idx_high_dist = np.argmax(min_dist_cand)
next_x_high_dist = np.asarray(candidate_pts[idx_high_dist]).ravel()
next_x_high_dist = np.clip(next_x_high_dist, 0.0, 0.999999)

# Choose one to submit (exploit / explore / high-distance)
next_x = next_x_exploit
print("Suggested next x (exploit):", next_x)
print("Alternative (explore):", next_x_explore)
print("Alternative (high distance to obs):", next_x_high_dist)


### Do the acquisition functions suggest similar points?
Pairwise distances below: small values mean two strategies agree; large values mean they point to different regions.

In [ ]:
# Compare suggested points (RBF acquisition + high-distance baseline)
names = ['EI', 'UCB', 'PI', 'Thompson', 'Entropy', 'High dist']
pts = [np.asarray(x).ravel() for x in [x_best_EI_RBF, x_best_UCB_RBF, x_best_PI_RBF, x_thomson_RBF, x_entropy_RBF, next_x_high_dist]]
n_pts = len(pts)
dists = np.zeros((n_pts, n_pts))
for i in range(n_pts):
    for j in range(n_pts):
        dists[i, j] = np.linalg.norm(pts[i] - pts[j])
print("Pairwise L2 distances between acquisition suggestions:")
for i in range(n_pts):
    for j in range(i + 1, n_pts):
        print(f"  {names[i]} vs {names[j]}: {dists[i, j]:.4f}")
max_d = dists.max()
if max_d < 0.15:
    print("\n>>> Suggestions are similar (max pairwise distance < 0.15).")
else:
    print(f"\n>>> Suggestions differ (max pairwise distance = {max_d:.4f}); consider comparing strategies.")

## 4. Illustrate the locations on the proposed query

In [ ]:
# PLOT ALL ACQUISITION FUNCTIONS + BASELINE (exploit / explore / high distance from cell above)
# Acquisition and high-distance points are from uniform-coverage candidates (CANDIDATE_SAMPLING_METHOD)

_method = CANDIDATE_SAMPLING_METHOD
X_best_combined = np.array([x_best_EI_RBF, x_best_UCB_RBF, x_best_PI_RBF, x_thomson_RBF, x_entropy_RBF])
labels_combined = [f'EI RBF ({_method})', f'UCB RBF ({_method})', f'PI RBF ({_method})', f'Thompson RBF ({_method})', f'Entropy RBF ({_method})']
colors_combined = plt.cm.cool(np.linspace(0, 1, len(X_best_combined)))

fig, ax = plt.subplots(figsize=(9, 7))
cf = ax.contourf(X1g, X2g, Y_grid, levels=22, cmap="turbo", alpha=0.8)
add_colorbar(cf, ax=ax, label="y (interpolated)", font_size_axis=DEFAULT_FONT_SIZE_AXIS)
ax.scatter(X[:, 0], X[:, 1], c=y, s=80, cmap="inferno", edgecolors="k", linewidths=1.5, zorder=5, label="Observations")
for i, (pt, lbl) in enumerate(zip(X_best_combined, labels_combined)):
    ax.scatter(pt[0], pt[1], c=[colors_combined[i]], s=80, marker='s', edgecolors="k", linewidths=1.5, zorder=10, alpha=0.95, label=lbl)
    ax.annotate(lbl, (pt[0], pt[1]), xytext=(5, 5), textcoords="offset points", fontsize=DEFAULT_FONT_SIZE_AXIS, alpha=0.9, zorder=11)
# Baseline suggestions (from cell above: exploit, explore, high distance to obs)
ax.scatter(next_x[0], next_x[1], c='k', s=80, marker='^', edgecolors='w', linewidths=1.5, zorder=10, alpha=0.95, label='Naive exploit')
ax.annotate('Naive exploit', (next_x[0], next_x[1]), xytext=(5, 5), textcoords="offset points", fontsize=DEFAULT_FONT_SIZE_AXIS, alpha=0.9, zorder=11)
ax.scatter(next_x_explore[0], next_x_explore[1], c='dimgray', s=80, marker='v', edgecolors='w', linewidths=1.5, zorder=10, alpha=0.95, label='Random explore')
ax.annotate('Random explore', (next_x_explore[0], next_x_explore[1]), xytext=(5, 5), textcoords="offset points", fontsize=DEFAULT_FONT_SIZE_AXIS, alpha=0.9, zorder=11)
# New method: uniform-coverage suggestion — point farthest from observations on candidate set
ax.scatter(next_x_high_dist[0], next_x_high_dist[1], c='blue', s=80, marker='v', edgecolors='k', linewidths=1.5, zorder=10, alpha=0.95, label=f'Highest distance (uniform-coverage/{_method})')
ax.annotate(f'Uniform-coverage ({_method})', (next_x_high_dist[0], next_x_high_dist[1]), xytext=(5, 5), textcoords="offset points", fontsize=DEFAULT_FONT_SIZE_AXIS, alpha=0.9, zorder=11)
style_axis(ax, xlabel="x_1", ylabel="x_2", title=f"All acquisition next points + baseline (acquisition over {_method} candidates)", font_size_axis=DEFAULT_FONT_SIZE_AXIS, font_size_titles=DEFAULT_FONT_SIZE_TITLES)
ax.legend(loc="upper center", bbox_to_anchor=(0.5, -0.12), fontsize=DEFAULT_FONT_SIZE_AXIS, ncol=4, frameon=True)
ax.set_aspect("equal")
plt.tight_layout()
fig.subplots_adjust(bottom=0.22)
if IF_SHOW_PLOT:
    plt.show()
if IF_EXPORT_PLOT:
    out_dir = PLOT_EXPORT_DIR
    assert_not_under_initial_data(out_dir, project_root=repo_root)
    out_dir.mkdir(parents=True, exist_ok=True)
    out_path = out_dir / f"function_1_all_acquisition_points.{DEFAULT_EXPORT_FORMAT}"
    fig.savefig(out_path, dpi=DEFAULT_EXPORT_DPI, format=DEFAULT_EXPORT_FORMAT)
    print("Plot saved to", out_path)

## 5. Select next query

In [ ]:
# In this cell we select the next query to submit.
# F1: sparse hotspot — use high-distance (farthest from observations) to explore until we get a non-zero reading.
# Alternatives: x_best_UCB_RBF (explore with UCB), x_best_EI_RBF, x_thomson_RBF, next_x_exploit (once hotspot found).
next_x = next_x_high_dist  # farthest from observations (explore); switch to EI/UCB/exploit after improvement
next_x = np.clip(np.asarray(next_x).ravel(), 0.0, 0.999999)
print(f"Next query (high distance to obs): ({next_x[0]:.4f}, {next_x[1]:.4f})")


## 6. Append new feedback (after portal returns)

After you submit and receive the new **(x, y)** from the portal for Function 1, paste the values below and run this cell. It appends the new point to a local dataset under `data/problems/function_1/`. The next time you run the notebook from the top, section 1 will load from this local dataset (initial + all appended points).

In [ ]:
if not IF_APPEND_DATA:
    print("IF_APPEND_DATA is False; append skipped. Set IF_APPEND_DATA = True and paste portal feedback below to run.")
else:
    # Paths (use repo_root from setup cell)
    _local_dir = repo_root / "data" / "problems" / "function_1"
    assert_not_under_initial_data(_local_dir, project_root=repo_root)
    _csv_path = _local_dir / "observations.csv"

    # Set these from the portal feedback for Function 1 (one new input and one new output)
    x_new = np.array([0.472352, 0.625531], dtype=np.float64)  # replace with your submitted input
    y_new = -1.802e-144  # replace with the output returned (scalar)

    # Load current data (CSV if exists, else initial_data). Under data/ we use only CSV.
    if _csv_path.exists():
        X_cur, y_cur = load_problem_data_csv(_csv_path)
    else:
        X_cur, y_cur = load_function_data(function_id=1)

    # Append: one new row for inputs, one new value for outputs
    x_new = np.atleast_2d(x_new)
    X_updated = np.vstack((X_cur, x_new))
    y_updated = np.append(y_cur, y_new)

    # Save to local directory (do not overwrite read-only initial_data)
    _local_dir.mkdir(parents=True, exist_ok=True)
    save_problem_data_csv(_local_dir / "observations.csv", X_updated, y_updated)
    print("Appended. Total points:", len(y_updated))
    print("Saved to", _local_dir, "-> observations.csv")

## 7. Save suggestion for submission

Write the chosen x to `data/submissions/` so you can upload it to the portal. **Portal format:** `x1-x2-...-xn` with **exactly six decimal places**, values in [0, 0.999999], **hyphens only, no spaces** (e.g. `0.498317-0.625531`). After you receive the new y, run section 6 (Append new feedback) to add it to your dataset, then re-run the notebook for the next round.

In [ ]:
if IF_EXPORT_QUERIES:
    # Portal format: each value 0.XXXXXX (exactly 6 decimals), in [0, 0.999999], hyphen-separated, no spaces
    next_x_clip = np.clip(np.asarray(next_x, dtype=np.float64).ravel(), 0.0, 0.999999)
    portal_str = "-".join(f"{v:.6f}" for v in next_x_clip)
    out_dir = repo_root / "data" / "submissions" / "function_1"
    assert_not_under_initial_data(out_dir, project_root=repo_root)
    out_dir.mkdir(parents=True, exist_ok=True)
    np.save(out_dir / "next_input.npy", next_x_clip)
    (out_dir / "next_input_portal.txt").write_text(portal_str)
    print("Saved to", out_dir / "next_input.npy")
    print("Portal string (copy-paste to portal):", portal_str)
    print("Also saved to", out_dir / "next_input_portal.txt")
else:
    print("IF_EXPORT_QUERIES is False; next_input.npy not saved.")